In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# if in colab:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls /content/drive/MyDrive

'Colab Notebooks'   molecular_modeling	 QR_illness.pdf  'Таблица 1.2.5.gsheet'
 Data		    Notability		 Topic.gdoc


In [5]:
path = '/content/drive/MyDrive/Data/train.csv'

data = pd.read_csv(path)

data.shape

(891, 12)

In [6]:
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
data = data.loc[:, data.columns != ('Name' and 'Cabin')]

In [8]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

In [9]:
data = data.loc[data['Embarked'].notna()]

mean_age = round(data['Age'].mean(), 2)
mean_age

29.64

Бустинг требует, чтобы число объектов было >> числа признаков, поэтому прйдется отказаться от 'Ticket"

In [10]:
data['Age'] = data['Age'].fillna(mean_age)

In [11]:
num_cols = [
    'Pclass',
    'Age',
    'SibSp',
    'Parch',
    'Fare',

]

cat_cols = [
    'Sex',
    'Embarked',
    # 'Ticket'
]

target_col = 'Survived'

X = data[num_cols + cat_cols]
y = data[[target_col]].values.ravel()

Деревьям не нужна нормализация, они чисто логические алгоритмы - им все равно на распределение

In [12]:
!pip install catboost

In [13]:
import catboost

In [14]:
model = catboost.CatBoostClassifier(
    cat_features=cat_cols,
    eval_metric='Accuracy',
    loss_function='Logloss',
    task_type="GPU",
    silent=False
)

params = {      # params - это гиперпараметры самого классификатора
    'l2_leaf_reg': np.linspace(0, 1, 10),
    'n_estimators': np.linspace(100, 200, 5),
    # 'max_depth': range(3, 7),

}

model.grid_search(
    X=X,
    y=y,
    param_grid=params,
    search_by_train_test_split=False,        # Если датасет маленький, то тут лучше false
    cv=5,
    verbose=False
)

Streaming output truncated to the last 5000 lines.
74:	learn: 0.8523207	test: 0.8258427	best: 0.8314607 (15)	total: 4.15s	remaining: 6.91s
75:	learn: 0.8481013	test: 0.8258427	best: 0.8314607 (15)	total: 4.21s	remaining: 6.86s
76:	learn: 0.8495077	test: 0.8258427	best: 0.8314607 (15)	total: 4.26s	remaining: 6.81s
77:	learn: 0.8579466	test: 0.8258427	best: 0.8314607 (15)	total: 4.32s	remaining: 6.76s
78:	learn: 0.8579466	test: 0.8258427	best: 0.8314607 (15)	total: 4.36s	remaining: 6.68s
79:	learn: 0.8565401	test: 0.8202247	best: 0.8314607 (15)	total: 4.42s	remaining: 6.63s
80:	learn: 0.8565401	test: 0.8202247	best: 0.8314607 (15)	total: 4.48s	remaining: 6.58s
81:	learn: 0.8579466	test: 0.8146067	best: 0.8314607 (15)	total: 4.53s	remaining: 6.53s
82:	learn: 0.8579466	test: 0.8146067	best: 0.8314607 (15)	total: 4.59s	remaining: 6.48s
83:	learn: 0.8593530	test: 0.8202247	best: 0.8314607 (15)	total: 4.66s	remaining: 6.43s
84:	learn: 0.8593530	test: 0.8146067	best: 0.8314607 (15)	total: 4.72

{'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
               47,
               48,
               49,
             

How to save and load model in catboost: https://stackoverflow.com/questions/51895761/how-to-correctly-load-pretrained-model-in-catboost-in-python

In [15]:
model.save_model('model.cbm')

In [21]:
model.fit(X, y)

0:	learn: 0.8301462	total: 47ms	remaining: 4.65s
1:	learn: 0.8346457	total: 97.4ms	remaining: 4.77s
2:	learn: 0.8312711	total: 130ms	remaining: 4.21s
3:	learn: 0.8335208	total: 156ms	remaining: 3.73s
4:	learn: 0.8335208	total: 174ms	remaining: 3.31s
5:	learn: 0.8335208	total: 224ms	remaining: 3.51s
6:	learn: 0.8346457	total: 252ms	remaining: 3.34s
7:	learn: 0.8380202	total: 284ms	remaining: 3.27s
8:	learn: 0.8323960	total: 297ms	remaining: 3s
9:	learn: 0.8312711	total: 312ms	remaining: 2.81s
10:	learn: 0.8312711	total: 328ms	remaining: 2.65s
11:	learn: 0.8346457	total: 348ms	remaining: 2.55s
12:	learn: 0.8368954	total: 364ms	remaining: 2.43s
13:	learn: 0.8357705	total: 401ms	remaining: 2.46s
14:	learn: 0.8380202	total: 432ms	remaining: 2.45s
15:	learn: 0.8413948	total: 471ms	remaining: 2.47s
16:	learn: 0.8346457	total: 504ms	remaining: 2.46s
17:	learn: 0.8380202	total: 540ms	remaining: 2.46s
18:	learn: 0.8357705	total: 581ms	remaining: 2.48s
19:	learn: 0.8323960	total: 596ms	remaining:

In [45]:
path_test = '/content/drive/MyDrive/Data/test.csv'
data_test = pd.read_csv(path_test)

data_test['Age'] = data_test['Age'].fillna(mean_age)    # среднее значение берем из обучающей выборки

X_test = data_test[num_cols + cat_cols]

X_test.isna().sum()

Pclass      0
Age         0
SibSp       0
Parch       0
Fare        1
Sex         0
Embarked    0
dtype: int64

In [46]:
mean_Fare = data['Fare'].mean()   # Чтобы заполнить один пропуск в тестовых данных, я используя среднее значение из трейновых

X_test['Fare'] = X_test['Fare'].fillna(mean_Fare)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [49]:
y_pred = model.predict(X_test)

In [51]:
subm_path = '/content/drive/MyDrive/Data/gender_submission.csv'

subm_df = pd.read_csv(subm_path, index_col='PassengerId')
subm_df['Survived'] = y_pred

subm_df.to_csv('boosting_submission.csv')